In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
from helper import *
import glob

In [3]:
loc_data = '/Users/yuxuan/Desktop/kaggle_optiver/optiver-realized-volatility-prediction'

In [39]:
list_order_book_file_train

['/Users/yuxuan/Desktop/kaggle_optiver/optiver-realized-volatility-prediction/book_train.parquet/stock_id=17',
 '/Users/yuxuan/Desktop/kaggle_optiver/optiver-realized-volatility-prediction/book_train.parquet/stock_id=28',
 '/Users/yuxuan/Desktop/kaggle_optiver/optiver-realized-volatility-prediction/book_train.parquet/stock_id=10',
 '/Users/yuxuan/Desktop/kaggle_optiver/optiver-realized-volatility-prediction/book_train.parquet/stock_id=26',
 '/Users/yuxuan/Desktop/kaggle_optiver/optiver-realized-volatility-prediction/book_train.parquet/stock_id=19',
 '/Users/yuxuan/Desktop/kaggle_optiver/optiver-realized-volatility-prediction/book_train.parquet/stock_id=21',
 '/Users/yuxuan/Desktop/kaggle_optiver/optiver-realized-volatility-prediction/book_train.parquet/stock_id=75',
 '/Users/yuxuan/Desktop/kaggle_optiver/optiver-realized-volatility-prediction/book_train.parquet/stock_id=81',
 '/Users/yuxuan/Desktop/kaggle_optiver/optiver-realized-volatility-prediction/book_train.parquet/stock_id=86',
 

In [6]:
list_order_book_file_train = glob.glob(f'{loc_data}/book_train.parquet/*')
list_trade_file_train = glob.glob(f'{loc_data}/trade_train.parquet/*')

In [7]:
df0_order_book = pd.read_parquet(list_order_book_file_train[0])
df0_trade = pd.read_parquet(list_trade_file_train[0])

In [10]:
df0_all = df0_order_book.merge(df0_trade, 
                               on = ['time_id', 'seconds_in_bucket'],
                               how = 'left'
                              )

In [8]:
df0_order_book.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.001186,1.001581,1.001044,1.001750,100,200,63,100
1,5,2,1.001186,1.001581,1.001016,1.001750,100,100,63,100
2,5,4,1.001186,1.001750,1.001016,1.002032,176,100,63,2
3,5,5,1.001355,1.001750,1.001186,1.001891,100,100,76,63
4,5,6,1.001355,1.001750,1.001186,1.002032,200,100,76,2


In [9]:
df0_trade.head()

,time_id,seconds_in_bucket,price,size,order_count
0,5,4,1.001581,100,2
1,5,17,1.001369,400,4
2,5,34,1.001440,200,2
3,5,42,1.001422,126,5
4,5,43,1.001553,100,2


In [11]:
len(df0_order_book), len(df0_trade), len(df0_all)

(1514773, 274600, 1514773)

In [12]:
 len(df0_trade)/len(df0_order_book)

0.18128128769129104

In [13]:
df0_all.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,price,size,order_count
0,5,0,1.001186,1.001581,1.001044,1.001750,100,200,63,100,NaN,NaN,NaN
1,5,2,1.001186,1.001581,1.001016,1.001750,100,100,63,100,NaN,NaN,NaN
2,5,4,1.001186,1.001750,1.001016,1.002032,176,100,63,2,1.001581,100.0,2.0
3,5,5,1.001355,1.001750,1.001186,1.001891,100,100,76,63,NaN,NaN,NaN
4,5,6,1.001355,1.001750,1.001186,1.002032,200,100,76,2,NaN,NaN,NaN


The features to build should be a result of aggregation. That is, for each time id and stock pair, we need to aggregate the data of different seconds in a bucket. The number of seconds is fixed as 600 for order book data, but a much smaller unfixed number for trade data.

1. Can/Should the aggregation take the time lag into consideration? Something like volatility from weighted average of WAP.
2. Craft features 

In [26]:
df_book_data = df0_order_book
wap1 = get_wap(bid_p = df_book_data['bid_price1'], 
                                  bid_s = df_book_data['bid_size1'], 
                                  ask_p = df_book_data['ask_price1'], 
                                  ask_s = df_book_data['ask_size1'])

wap2 = get_wap_from_df(df0_order_book, 
                       bid_p = 'bid_price1', bid_s = 'bid_size1',
                       ask_p = 'ask_price1', ask_s = 'ask_size1'
                      )

(wap1 == wap2).all()

True

In [38]:
logr_1 = log_return_per_time_id(list_order_book_file_train[0])
logr_1 = logr_1['log_return']

logr_2 = get_log_return_from_df(df0_order_book, 
                       bid_p = 'bid_price1', bid_s = 'bid_size1',
                       ask_p = 'ask_price1', ask_s = 'ask_size1'
                      )

(logr_1 == logr_2).all()

True